# Inference

This script showcases the different models available in fishbAIT and how to use them efficiently.

In [4]:
import sys

sys.path

['/cephyr/users/skall/Alvis/Ecotoxformer',
 '',
 '/apps/Arch/software/plotly.py/4.14.3-GCCcore-10.2.0/lib/python3.8/site-packages',
 '/apps/Arch/software/Pillow/8.0.1-GCCcore-10.2.0/lib/python3.8/site-packages',
 '/apps/Arch/software/JupyterLab/2.2.8-GCCcore-10.2.0/lib/python3.8/site-packages',
 '/apps/Arch/software/IPython/7.18.1-GCCcore-10.2.0/lib/python3.8/site-packages',
 '/apps/Arch/software/h5py/3.1.0-fosscuda-2020b/lib/python3.8/site-packages',
 '/apps/Arch/software/matplotlib/3.3.3-fosscuda-2020b/lib/python3.8/site-packages',
 '/apps/Arch/software/Tkinter/3.8.6-GCCcore-10.2.0/lib/python3.8/site-packages',
 '/apps/Arch/software/scikit-learn/0.23.2-fosscuda-2020b/lib/python3.8/site-packages',
 '/apps/Arch/software/SciPy-bundle/2020.11-fosscuda-2020b/lib/python3.8/site-packages',
 '/apps/Arch/software/PyTorch/1.9.0-fosscuda-2020b/lib/python3.8/site-packages',
 '/apps/Arch/software/typing-extensions/3.7.4.3-GCCcore-10.2.0/lib/python3.8/site-packages',
 '/apps/Arch/software/pybind11